In [2]:
import os
os.getcwd()

'C:\\Users\\U6080396'

In [3]:
import numpy
filename="CoEPrA.csv"
raw_data=open(filename,'rt')
data=numpy.loadtxt(raw_data,delimiter=",")

In [4]:
data.shape

(89, 5788)

In [12]:
X=data[:,:5787]
Y=data[:,5787]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X,Y, test_size=0.2, random_state=0)
Y_test.shape

(18,)

In [23]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [26]:
LinearRegr=linear_model.LinearRegression()
LinearRegr.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
predictions=LinearRegr.predict(X_train)

In [20]:
print(mean_squared_error(Y_train,predictions)) # overfitting

0.006776305247293394


In [29]:
# K-fold cross validation
score=cross_val_score(LinearRegr, X_train, Y_train, scoring='neg_mean_squared_error', cv=5)
print(score)
print(np.mean(score))

[-2.77162774e+15 -1.63780071e+23 -9.01357349e+17 -1.89420654e+17
 -1.53857191e+16]
-3.2756236040128235e+22


In [34]:
pred_test=LinearRegr.predict(X_test)
print(mean_squared_error(Y_test,pred_test))

1.1313658797668658e+24


In [63]:
#Lasso (cost function includes penalty for features propotional to sum of absolute value of features instead of sum of its squares)
lasso=linear_model.Lasso(alpha=0.3, max_iter=1000000)
lasso.fit(X_train,Y_train)

Lasso(alpha=0.3, copy_X=True, fit_intercept=True, max_iter=1000000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [64]:
pred_train_lasso=lasso.predict(X_train)
pred_test_lasso=lasso.predict(X_test)

In [65]:
print('Lasso MSE train:', mean_squared_error(Y_train,pred_train_lasso))
print('Lasso MSE test:', mean_squared_error(Y_test,pred_test_lasso))

Lasso MSE train: 0.030990182372048328
Lasso MSE test: 0.5542167080273583


In [100]:
score_lasso=cross_val_score(lasso, X_train, Y_train, scoring='neg_mean_squared_error', cv=5)
print(np.mean(score_lasso)) 

-0.7746651811184468


In [45]:
print(lasso.coef_)
print(np.nonzero(lasso.coef_))

[-0.  0. -0. ... -0. -0.  0.]
(array([ 117,  136,  445,  446,  715,  787, 1088, 1094, 1236, 1422, 1430,
       1732, 1737, 1874, 1938, 2247, 2380, 2581, 2708, 3023, 3160, 3224,
       3351, 3533, 3666, 3994, 4176, 4221, 4303, 4309, 4510, 4511, 4637,
       4819, 4947, 4952, 5216, 5462, 5589, 5735, 5737], dtype=int64),)


In [82]:
non_zero_coef=np.nonzero(lasso.coef_)
X_train_filtered=X_train[:,non_zero_coef[0]]

X_test_filtered=X_test[:,non_zero_coef[0]]
X_train_filtered.shape #49 features that impact result
X_test_filtered.shape

(18, 49)

In [72]:
#Ridge regression penalyzes model with bias propotional to sum of squares of weihts L2
ridge=linear_model.Ridge(alpha=0.8,max_iter=1000000)
ridge.fit(X_train_filtered,Y_train)

Ridge(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=1000000,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [95]:
ridge_pred=ridge.predict(X_train_filtered)
ridge_pred_test=ridge.predict(X_test_filtered)

print('MSE ridge train:',mean_squared_error(Y_train,ridge_pred))
print('MSE ridge test:',mean_squared_error(Y_test,ridge_pred_test))

MSE ridge train: 0.016127335916395535
MSE ridge test: 0.987849520725775


In [99]:
score_ridge=cross_val_score(ridge, X_test_filtered, Y_test, scoring='neg_mean_squared_error', cv=5)
print(np.mean(score_ridge)) #Still overfitting

-2.114622424644933
